#Introduction:
This notes is for SQL optimization, performance analysis, and working with a cloud database using Google Colab.

##Objectives:
- Optimize SQL queries for better performance.
- Analyze and improve view performance.
- Connect to and interact with a cloud-based database.




#1. Setting Up Your Environment
Before you begin, we'll need to install the necessary Python libraries to interact with your database. Run the following command:

In [ ]:
!pip install mysql-connector-python faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 35.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


#2. Database Connection and Data Generation
We have to connect with our database credentials to establish a connection to your cloud database.

We've provided a Python script to help you populate  tables with more records. Execute the script as shown below.

In [ ]:
import psycopg2
import random
from faker import Faker

fake = Faker()

# Connect to your database
import mysql.connector as mysql
conn = mysql.connect(
    host='dbclass-database.cqmswswxnp9n.us-east-1.rds.amazonaws.com',  # replace this with your host url
    database='EClaim',
    user='dbstudent',
    password='opimdatabase',
)

conn.autocommit = False
cursor = conn.cursor()

# Function to insert fake data into PatientInformation table
def create_fake_patient_information(n):
    patient_data = [
        (
            fake.unique.uuid4(),
            fake.first_name(),
            fake.last_name(),
            fake.date_of_birth(),
            fake.random_element(elements=('Male', 'Female')),
            fake.street_address(),
            fake.phone_number(),
            fake.email(),
            fake.unique.uuid4()
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO PatientInformation (PatientID, FirstName, LastName, DateOfBirth, Gender, Address, PhoneNumber, EmailAddress, InsurancePlanID) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        patient_data
    )
    conn.commit()

# Function to insert fake data into Encounter table
def create_fake_encounter(n):
    encounter_data = [
        (
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.date_this_decade(),
            fake.time(),
            fake.sentence(),
            fake.random_element(elements=('Outpatient', 'Inpatient')),
            fake.random_element(elements=('Billed', 'Not Billed'))
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO Encounter (EncounterID, PatientID, ProviderID, DateOfEncounter, TimeOfEncounter, ChiefComplaint, VisitType, BillingStatus) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
        encounter_data
    )
    conn.commit()

# Function to insert fake data into Diagnosis table
def create_fake_diagnosis(n):
    diagnosis_data = [
        (
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.random_element(elements=('ICD-10-001', 'ICD-10-002', 'ICD-10-003')),
            fake.sentence(),
            fake.date_this_decade()
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO Diagnosis (DiagnosisID, EncounterID, MappedDiagnosisCode, DiagnosisDescription, DateOfDiagnosis) VALUES (%s, %s, %s, %s, %s)",
        diagnosis_data
    )
    conn.commit()

# Function to insert fake data into ProcedureInfo table
def create_fake_procedure_info(n):
    procedure_data = [
        (
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.random_element(elements=('CPT-001', 'CPT-002', 'CPT-003')),
            fake.sentence(),
            fake.date_this_decade(),
            fake.random_number(digits=5)
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO ProcedureInfo (ProcedureID, EncounterID, MappedProcedureCode, ProcedureDescription, ProcedureDate, ProcedureFee) VALUES (%s, %s, %s, %s, %s, %s)",
        procedure_data
    )
    conn.commit()

# Function to insert fake data into InsuranceInformation table
def create_fake_insurance_information(n):
    insurance_data = [
        (
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.company(),
            fake.date_this_decade(),
            fake.date_this_decade(),
            fake.random_number(digits=2),
            fake.random_number(digits=4),
            fake.random_number(digits=5)
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO InsuranceInformation (InsurancePlanID, PatientID, PolicyNumber, PayerName, CoverageStartDate, CoverageEndDate, CoPaymentAmount, DeductibleAmount, OutOfPocketMaximum) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
        insurance_data
    )
    conn.commit()

# Function to insert fake data into Claim table
def create_fake_claim(n):
    claim_data = [
        (
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            [fake.random_element(elements=('ICD-10-001', 'ICD-10-002', 'ICD-10-003')) for _ in range(3)],
            [fake.random_element(elements=('CPT-001', 'CPT-002', 'CPT-003')) for _ in range(3)],
            fake.random_element(elements=('Submitted', 'In Review')),
            fake.random_number(digits=5),
            fake.date_this_decade(),
            fake.date_this_decade()
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO Claim (ClaimID, EncounterID, PatientID, InsurancePlanID, DiagnosisCodes, ProcedureCodes, ClaimStatus, ClaimAmount, SubmissionDate, PaymentDate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
        claim_data
    )
    conn.commit()

# Function to insert fake data into ProviderInfo table
def create_fake_provider_info(n):
    provider_data = [
        (
            fake.unique.uuid4(),
            fake.company(),
            fake.random_element(elements=('Physician', 'Specialist')),
            fake.unique.uuid4(),
            fake.unique.uuid4(),
            fake.street_address(),
            fake.phone_number()
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO ProviderInfo (ProviderID, ProviderName, ProviderType, ProviderNPINumber, TaxIDNumber, ProviderAddress, ContactInformation) VALUES (%s, %s, %s, %s, %s, %s, %s)",
        provider_data
    )
    conn.commit()

# Function to insert fake data into ClaimEncounter table
def create_fake_claim_encounter(n):
    claim_encounter_data = [
        (
            fake.unique.uuid4(),
            fake.unique.uuid4()
        )
        for _ in range(n)
    ]
    cursor.executemany(
        "INSERT INTO ClaimEncounter (ClaimID, EncounterID) VALUES (%s, %s)",
        claim_encounter_data
    )
    conn.commit()

# Function to insert fake data into Diagnosiscodes table
def create_fake_diagnosis_codes(n):
    diagnosis_codes_data = [
        (
            fake.random_element(elements=('ICD-10-001', 'ICD-10-002', 'ICD-10-003')),
            fake.sentence()
        )
        for _ in range(n)
    ]
conn.commit()